<a href="https://colab.research.google.com/github/toskpl/dw_matrix_road_sing/blob/master/day5_matrix_part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install hyperopt

In [0]:
"""import bibliotek"""
import pandas as pd
import numpy as np


import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout
from tensorflow.keras.utils import to_categorical

#do wizualizacjai krzywej uczenia sie
%load_ext tensorboard

import matplotlib.pyplot as plt

from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK,tpe,Trials,fmin

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/DW_Matrix/matrix_three/dw_matrix_road_sing'

/content/drive/My Drive/Colab Notebooks/DW_Matrix/matrix_three/dw_matrix_road_sing


In [0]:
"""wczytane danych z pliki typu pickle"""
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')


"""rozmiar zbioru danych"""
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']
X_train.shape,y_train.shape,X_test.shape,y_test.shape


((34799, 32, 32, 3), (34799,), (4410, 32, 32, 3), (4410,))

In [0]:
"""transformacja jesli jest 1 to przypisz kategorie mozna uruchamiac wiele razy"""
if y_train.ndim == 1 : y_train = to_categorical(y_train)
if y_test.ndim == 1 : y_test = to_categorical(y_test)

In [0]:
"""pomocniczne zmianne do modelu jako wesjcieowe - zachouwje rozmiary wektorow"""
input_shape = X_train.shape [1:]
num_classes = y_train.shape[1]

In [0]:
"""funkcje pomocnicze"""
def train_model(model, X_train,y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs",datetime.datetime.now().strftime("%Y-%m-%d-%H%M%S"))
  tensorboard_callbacks = tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)

  model.fit(X_train,
            y_train,
            batch_size = params_fit.get('batch_size',128),
            epochs = params_fit.get('epochs',5),
            verbose = params_fit.get('verbose',1),
            validation_data = params_fit.get('validation_data',(X_train,y_train)),
            callbacks = [tensorboard_callbacks]
  )

  return model

def predict(model_trained, X_test,y_test,scoring=accuracy_score):
    y_test_norm = np.argmax(y_test,axis=1)

    y_predict_prob = model_trained.predict(X_test)
    y_pred = np.argmax(y_predict_prob,axis=1)
    return scoring(y_test_norm,y_pred)
  

def train_and_predict(model):
  model_trained = train_model(model,X_train,y_train)
  return predict(model_trained,X_test,y_test)

In [0]:
"""model"""
def gen_cnn_v5(input_shape,num_classes):
  return Sequential([
         Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=input_shape ),
         Conv2D(filters=32,kernel_size=(3,3),activation='relu' ,padding='same'),
         MaxPool2D(),
         Dropout(0.3),

         Conv2D(filters=64,kernel_size=(3,3),activation='relu' ,padding='same' ),
         Conv2D(filters=64,kernel_size=(3,3),activation='relu' ),
         MaxPool2D(),
         Dropout(0.3),

         Conv2D(filters=64,kernel_size=(3,3),activation='relu' ,padding='same' ),
         Conv2D(filters=64,kernel_size=(3,3),activation='relu' ),
         MaxPool2D(),
         Dropout(0.3),

         Flatten(),
         Dense(1024,activation='relu'),
         Dropout(0.3),

         Dense(1024,activation='relu'),
         Dropout(0.3),
         Dense(num_classes,activation='softmax')


])
  
train_and_predict( gen_cnn_v5(input_shape,num_classes))

Epoch 1/5
272/272 [==============================] - 5s 19ms/step - loss: 2.5487 - accuracy: 0.3103 - val_loss: 0.9428 - val_accuracy: 0.7377
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 0.8733 - accuracy: 0.7222 - val_loss: 0.2358 - val_accuracy: 0.9274
Epoch 3/5
272/272 [==============================] - 5s 17ms/step - loss: 0.4516 - accuracy: 0.8603 - val_loss: 0.1158 - val_accuracy: 0.9646
Epoch 4/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2914 - accuracy: 0.9093 - val_loss: 0.0591 - val_accuracy: 0.9835
Epoch 5/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2156 - accuracy: 0.9347 - val_loss: 0.0437 - val_accuracy: 0.9875


0.9609977324263038

In [0]:
model = gen_cnn_v5(input_shape,num_classes)
model_trained = train_model(model,X_train,y_train,params_fit={})
predict(model_trained,X_test,y_test)

Epoch 1/5
272/272 [==============================] - 5s 18ms/step - loss: 2.4394 - accuracy: 0.3248 - val_loss: 0.8733 - val_accuracy: 0.7457
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 0.8034 - accuracy: 0.7457 - val_loss: 0.2678 - val_accuracy: 0.9190
Epoch 3/5
272/272 [==============================] - 5s 17ms/step - loss: 0.4285 - accuracy: 0.8631 - val_loss: 0.1563 - val_accuracy: 0.9505
Epoch 4/5
272/272 [==============================] - 5s 17ms/step - loss: 0.3060 - accuracy: 0.9059 - val_loss: 0.0827 - val_accuracy: 0.9722
Epoch 5/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2313 - accuracy: 0.9277 - val_loss: 0.0504 - val_accuracy: 0.9833


0.9582766439909297

In [0]:
"""funkcja do hyperopt"""
def fun_obj(params):
  model =  gen_cnn_v5(input_shape,num_classes)

  model.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])
  model.fit(X_train,
            y_train,
            batch_size = int(params.get('batch_size',128)),
            epochs = 7,
            verbose = 0,
  )

  accuracy = model.evaluate(X_test,y_test)[0]
  print(params,'accuracy={}'.format(accuracy))

  return {'loss' : -accuracy, 'status': STATUS_OK,'model':model}

In [0]:
"""funkcja do hyperopt"""
space = {
    'batch_size': hp.quniform('batch_size',50,200,20)
}

best = fmin (
    fun_obj,
    space,
    tpe.suggest,
    1,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.1301 - accuracy: 0.9688

 17/138 [==>...........................]
 - ETA: 0s - loss: 0.1330 - accuracy: 0.9688

 30/138 [=====>........................]
 - ETA: 0s - loss: 0.1359 - accuracy: 0.9688

 43/138 [========>.....................]
 - ETA: 0s - loss: 0.1000 - accuracy: 0.9775

 56/138 [===========>..................]
 - ETA: 0s - loss: 0.1278 - accuracy: 0.9704

 71/138 [==============>...............]
 - ETA: 0s - loss: 0.1115 - accuracy: 0.9723


In [0]:
def get_model(params):
  return Sequential([
         Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=input_shape ),
         Conv2D(filters=32,kernel_size=(3,3),activation='relu' ,padding='same'),
         Conv2D(filters=32,kernel_size=(3,3),activation='relu' ,padding='same'),
         MaxPool2D(),
         Dropout(params['dropout_cnn_one']),

         Conv2D(filters=64,kernel_size=(3,3),activation='relu' ,padding='same' ),
         Conv2D(filters=64,kernel_size=(3,3),activation='relu' ,padding='same'),
         Conv2D(filters=64,kernel_size=(3,3),activation='relu' ),
         MaxPool2D(),
         Dropout(params['dropout_cnn_two']),

         Conv2D(filters=128,kernel_size=(3,3),activation='relu' ,padding='same' ),
         Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same' ),
         Conv2D(filters=128,kernel_size=(3,3),activation='relu' ),
         MaxPool2D(),
         Dropout(params['dropout_cnn_three']),

         Flatten(),
         Dense(1024,activation='relu'),
         Dropout(params['dropout_dense_one']),

         Dense(1024,activation='relu'),
         Dropout(params['dropout_dense_two']),
         Dense(num_classes,activation='softmax')


])

In [0]:
"""funkcja do hyperopt"""
def fun_obj(params):
  model =  get_model(params)

  model.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])
  model.fit(X_train,
            y_train,
            batch_size = int(params.get('batch_size',128)),
            epochs = 7,
            verbose = 0,
  )

  score = model.evaluate(X_test,y_test,verbose=0)
  accuracy = score[1]
  print(params,'accuracy={}'.format(accuracy))

  return {'loss' : -accuracy, 'status': STATUS_OK,'model':model}

In [0]:
"""funkcja do hyperopt"""
space = {
    'batch_size': hp.quniform('batch_size',100,200,10),
    'dropout_cnn_one':hp.uniform ('dropout_cnn_one',0.3,0.5),
    'dropout_cnn_two':hp.uniform ('dropout_cnn_two',0.3,0.5),
    'dropout_cnn_three':hp.uniform ('dropout_cnn_three',0.3,0.5),

    'dropout_dense_one':hp.uniform ('dropout_dense_one',0.3,0.7),
    'dropout_dense_two':hp.uniform ('dropout_dense_two',0.3,0.7)

}

best = fmin (
    fun_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 180.0, 'dropout_cnn_one': 0.33765946773273275, 'dropout_cnn_three': 0.3427054125868428, 'dropout_cnn_two': 0.36637419928344106, 'dropout_dense_one': 0.46973120419349434, 'dropout_dense_two': 0.6982790524843541}
accuracy=0.9539682269096375
{'batch_size': 170.0, 'dropout_cnn_one': 0.4074722125032846, 'dropout_cnn_three': 0.4141444718606718, 'dropout_cnn_two': 0.3224248334165675, 'dropout_dense_one': 0.5524017057540858, 'dropout_dense_two': 0.4244978490090039}
accuracy=0.9680272340774536
{'batch_size': 160.0, 'dropout_cnn_one': 0.34751464256278675, 'dropout_cnn_three': 0.3270349674299432, 'dropout_cnn_two': 0.32626067308950146, 'dropout_dense_one': 0.36332523950997325, 'dropout_dense_two': 0.5613989016276132}
accuracy=0.9702947735786438
{'batch_size': 180.0, 'dropout_cnn_one': 0.49452113260731734, 'dropout_cnn_three': 0.32797787926892863, 'dropout_cnn_two': 0.3057272310001801, 'dropout_dense_one': 0.3100224164579931, 'dropout_dense_two': 0.45432767980092814}
accuracy=0.9650

Udało się osiągnąć wynik : 0.9875283241271973
